In [97]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

In [98]:
download_url("http://files.fast.ai/data/cifar10.tgz", '.')
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')
classes = os.listdir( "./data/cifar10/train")

Using downloaded and verified file: .\cifar10.tgz


In [99]:
sts = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
t_tfms = tt.Compose([tt.RandomCrop(32, padding=4, padding_mode='reflect'), tt.RandomHorizontalFlip(), tt.ToTensor(), tt.Normalize(*sts,inplace=True)])
v_tfms = tt.Compose([tt.ToTensor(), tt.Normalize(*sts)])
t_ds = ImageFolder('./data/cifar10//train', t_tfms)
v_ds = ImageFolder('./data/cifar10//test', v_tfms)
t_dl = DataLoader(t_ds, 400, shuffle=True, num_workers=3, pin_memory=True)
v_dl = DataLoader(v_ds, 400*2, num_workers=3, pin_memory=True)

In [100]:
def get_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [101]:
def swapDevice(data, device):
    if isinstance(data, (list,tuple)):
        return [swapDevice(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [102]:
class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl: 
            yield swapDevice(b, self.device)

    def __len__(self):
        return len(self.dl)

In [103]:
t_dl = DeviceDataLoader(t_dl, get_device())
v_dl = DeviceDataLoader(v_dl, get_device())

In [104]:
class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        return self.relu2(out) + x

In [105]:
simple_resnet = swapDevice(SimpleResidualBlock(), get_device())

for images, labels in t_dl:
    out = simple_resnet(images)
    print(out.shape)
    break
    
del simple_resnet, images, labels
torch.cuda.empty_cache()

torch.Size([400, 3, 32, 32])


In [106]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [107]:
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  
        loss = F.cross_entropy(out, labels) 
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    
        loss = F.cross_entropy(out, labels)   
        acc = accuracy(out, labels)           
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))

In [108]:
def conv(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)


In [109]:
class ResNet9(ImageClassificationBase):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        
        self.conv1 = conv(in_channels, 64)
        self.conv2 = conv(64, 128, pool=True)
        self.res1 = nn.Sequential(conv(128, 128), conv(128, 128))
        
        self.conv3 = conv(128, 256, pool=True)
        self.conv4 = conv(256, 512, pool=True)
        self.res2 = nn.Sequential(conv(512, 512), conv(512, 512))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4), nn.Flatten(), nn.Linear(512, num_classes))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out

In [110]:
model = swapDevice(ResNet9(3, 10), get_device())

In [111]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


In [112]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [113]:
def foc(number_of_epochs, lr, model, train_loader, val_loader, weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    h = []

    opt = opt_func(model.parameters(), lr, weight_decay=weight_decay)
    sc = torch.optim.lr_scheduler.OneCycleLR(opt, lr, epochs=number_of_epochs,  steps_per_epoch=len(train_loader))
    
    for epoch in range(number_of_epochs):
        model.train()
        ls = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            ls.append(loss)
            loss.backward()
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)    
            opt.step()
            opt.zero_grad()
            lrs.append(get_lr(opt))
            sc.step()

        r = evaluate(model, val_loader)
        r['train_loss'] = torch.stack(ls).mean().item()
        r['lrs'] = lrs
        model.epoch_end(epoch, r)
        h.append(r)
    return h


In [114]:
h = [evaluate(model, v_dl)]

In [115]:
opt_func = torch.optim.Adam
weight_decay = 1e-4
n_o_epochs = 30
m_lr = 0.01
grad_clip = 0.1

In [117]:
%%time
h += foc(n_o_epochs, m_lr, model, t_dl, v_dl, grad_clip=grad_clip, weight_decay=weight_decay, opt_func=opt_func)

Epoch [0], last_lr: 0.00069, train_loss: 0.7987, val_loss: 0.6950, val_acc: 0.7522


KeyboardInterrupt: ignored